In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
cd

C:\Users\37789


In [3]:
df = pd.read_csv("flightPrice.csv")

In [4]:
df

,Date_of_journey,Journey_day,Airline,Flight_code,Class,Source,Departure,Total_stops,Arrival,Destination,Duration_in_hours,Days_left,Fare
0,2023-01-16,Monday,SpiceJet,SG-8169,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5335
1,2023-01-16,Monday,Indigo,6E-2519,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.3333,1,5899
2,2023-01-16,Monday,GO FIRST,G8-354,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.1667,1,5801
3,2023-01-16,Monday,SpiceJet,SG-8709,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5794
4,2023-01-16,Monday,Air India,AI-805,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.1667,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452083,2023-03-06,Monday,Vistara,UK-926,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,13.0833,50,65028
452084,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452085,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452086,2023-03-06,Monday,Vistara,UK-946,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,11.1667,50,72980


In [5]:
df.isnull().sum()

Date_of_journey      0
Journey_day          0
Airline              0
Flight_code          0
Class                0
Source               0
Departure            0
Total_stops          0
Arrival              0
Destination          0
Duration_in_hours    0
Days_left            0
Fare                 0
dtype: int64

In [6]:
df['Date_of_journey'] = pd.to_datetime(df['Date_of_journey'], format='%Y-%m-%d')
df["Day"] = df["Date_of_journey"].dt.day
df["Month"] = df["Date_of_journey"].dt.month
df["Year"] = df["Date_of_journey"].dt.year
df.drop(columns=["Date_of_journey"], inplace=True)

In [7]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [8]:
columns_to_encode = ['Class', 'Source','Airline','Journey_day','Destination']

In [9]:
one_hot_encoded_data = encoder.fit_transform(df[columns_to_encode])

In [10]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

one_hot_encoded_df.index = df.index

df.drop(columns=columns_to_encode, inplace=True)

df = pd.concat([df, one_hot_encoded_df], axis=1)

In [11]:
label_encoder = LabelEncoder()
df["Total_stops_encoded"] = label_encoder.fit_transform(df["Total_stops"])
df["Total_stops"] = df["Total_stops_encoded"]

In [12]:
df.drop(columns=["Flight_code"], inplace=True)

In [13]:
df["Departure"].unique()

array(['After 6 PM', 'Before 6 AM', '12 PM - 6 PM', '6 AM - 12 PM'],
      dtype=object)

In [14]:
df["Arrival"].unique()

array(['After 6 PM', 'Before 6 AM', '6 AM - 12 PM', '12 PM - 6 PM'],
      dtype=object)

In [15]:
Time_Slots = ['After 6 PM', 'Before 6 AM', '12 PM - 6 PM', '6 AM - 12 PM']

In [16]:
encoder = OneHotEncoder(categories=[Time_Slots,Time_Slots], drop='first', sparse_output=False)

In [17]:
one_hot_encoded_data = encoder.fit_transform(df[['Arrival', 'Departure']])
feature_names = encoder.get_feature_names_out(['Arrival', 'Departure'])

In [18]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=feature_names)
one_hot_encoded_df.index = df.index

In [19]:
df.drop(columns=['Arrival', 'Departure'], inplace=True)

In [20]:
df = pd.concat([df, one_hot_encoded_df], axis=1)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 43 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Total_stops             452088 non-null  int32  
 1   Duration_in_hours       452088 non-null  float64
 2   Days_left               452088 non-null  int64  
 3   Fare                    452088 non-null  int64  
 4   Day                     452088 non-null  int32  
 5   Month                   452088 non-null  int32  
 6   Year                    452088 non-null  int32  
 7   Class_Economy           452088 non-null  float64
 8   Class_First             452088 non-null  float64
 9   Class_Premium Economy   452088 non-null  float64
 10  Source_Bangalore        452088 non-null  float64
 11  Source_Chennai          452088 non-null  float64
 12  Source_Delhi            452088 non-null  float64
 13  Source_Hyderabad        452088 non-null  float64
 14  Source_Kolkata      

In [22]:
X = df.drop(columns=['Fare'])
y = df['Fare']

In [23]:
# Initializing the linear regression model
model = LinearRegression()

In [44]:
rfe = RFE(model, n_features_to_select=15)  
rfe = rfe.fit(X, y)

In [45]:
# Getting the selected features
selected_features = X.columns[rfe.support_]

In [46]:
# Displaying the selected features
print("Selected Features: ", selected_features)

Selected Features:  Index(['Total_stops', 'Month', 'Class_Economy', 'Class_First',
       'Class_Premium Economy', 'Source_Delhi', 'Source_Kolkata',
       'Airline_AirAsia', 'Airline_StarAir', 'Airline_Vistara',
       'Destination_Chennai', 'Destination_Kolkata', 'Total_stops_encoded',
       'Arrival_Before 6 AM', 'Arrival_6 AM - 12 PM'],
      dtype='object')


In [47]:
# Creating a new feature set with the selected features
X_selected = X[selected_features]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [49]:
# Fitting the linear regression model
model.fit(X_train, y_train)

LinearRegression()

In [50]:
# Making predictions on the training set
y_train_pred = model.predict(X_train)

In [51]:
# Making predictions on the test set
y_test_pred = model.predict(X_test)

In [52]:
# Evaluating the model on the training set
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Evaluating the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

In [33]:
# RFE with 5 features

print("Training Set Metrics")
print("Mean Squared Error: ", mse_train)
print("R-squared: ", r2_train)

print("\nTest Set Metrics")
print("Mean Squared Error: ", mse_test)
print("R-squared: ", r2_test)

Training Set Metrics
Mean Squared Error:  78500576.56257501
R-squared:  0.8096256281815373

Test Set Metrics
Mean Squared Error:  79373507.76573414
R-squared:  0.8076572600712463


In [43]:
# RFE with 10 features

print("Training Set Metrics")
print("Mean Squared Error: ", mse_train)
print("R-squared: ", r2_train)

print("\nTest Set Metrics")
print("Mean Squared Error: ", mse_test)
print("R-squared: ", r2_test)

Training Set Metrics
Mean Squared Error:  65549665.74267158
R-squared:  0.8410333148480258

Test Set Metrics
Mean Squared Error:  65895310.80713717
R-squared:  0.840318451510175


In [53]:
# RFE with 15 features

print("Training Set Metrics")
print("Mean Squared Error: ", mse_train)
print("R-squared: ", r2_train)

print("\nTest Set Metrics")
print("Mean Squared Error: ", mse_test)
print("R-squared: ", r2_test)

Training Set Metrics
Mean Squared Error:  64320975.355069906
R-squared:  0.8440130529715106

Test Set Metrics
Mean Squared Error:  64661692.39901264
R-squared:  0.843307831107024


After using RFE, number of features as 15 is giving us max accuracy.